종로구 환경기상 데이터 전처리
-------------------------------------------------------------------------------
> 1. 측정소마다 저장된 데이터를 하나의 파일로 통합
> 2. 법정동이름을 현업의 편의상 지정된 코드로 바꿔줌
> 3. 결측치, 이상치 제거

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import glob
import os
directory = "./Desktop/dataset/bigcon/환경기상데이터/"

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
# 노트북 안에 그래프를 그리기 위해
%matplotlib inline

# 그래프에서 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일을 사용
plt.style.use('ggplot')

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

데이터 불러오기
-------------------------------------------------------------------------------
>  측정소마다 저장된 데이터를 하나의 파일로 통합하기 위하여 데이터를 불러와 concat하여줍니다.


In [3]:
read_file_jongro_lst = os.listdir('./Desktop/dataset/bigcon/환경기상데이터/종로구/')
read_df_jongro_lst = []
print ("read_file_jongro_lst: {}".format(read_file_jongro_lst))

for f in read_file_jongro_lst:
    if f == '.ipynb_checkpoints':
        pass
    else :
        df_temp = pd.read_csv(directory + '종로구/' + f)
        read_df_jongro_lst.append(df_temp)


read_file_jongro_lst: ['V10O1611251.csv', 'V10O1611722.csv', 'V01o1610468.csv', 'V10O1611684.csv', 'V10O1611645.csv', 'V10O1610567.csv', 'V10O1611255.csv', 'V10O1611151.csv', 'V10O1611623.csv', 'V10O1611145.csv', 'V10O1611634.csv', 'V10O1611220.csv', 'V10O1611750.csv', 'V10O1612113.csv', 'V10O1612106.csv', 'V10O1610252.csv', 'V10O1611170.csv', 'V10O1611172.csv', 'V10O1611173.csv', 'V10O1611639.csv', 'V10O1611258.csv', 'V10O1610542.csv', 'V10O1610543.csv', 'V10O1611887.csv', 'V10O1610540.csv', 'V10O1611289.csv', 'V10O1610544.csv', 'V10O1610545.csv', 'V10O1611698.csv', 'V10O1610546.csv', 'V10O1611658.csv']


In [4]:
jongro = pd.concat(read_df_jongro_lst)

In [5]:
jongro.head()

,tm,serial,flag,pm10,co2,vocs,noise,temp,humi,pm25,Unnamed: 10
0,201804010000,V10O1611251,1,54,-9999,-9999,41,14.7,55,27.0,NaN
1,201804010001,V10O1611251,1,42,-9999,-9999,47,14.7,55,21.0,NaN
2,201804010002,V10O1611251,1,59,-9999,-9999,41,14.6,56,30.0,NaN
3,201804010003,V10O1611251,1,41,-9999,-9999,44,14.6,56,20.0,NaN
4,201804010004,V10O1611251,1,54,-9999,-9999,43,14.6,56,27.0,NaN


날짜 칼럼 이름 변경
-------------------------------------------------------------------------------
>  데이터의 통합을 위하여 tm으로 된 칼럼을 Y/M/D/h별로 나누어 줍니다.

In [6]:
jongro["Year"] = jongro["tm"]
jongro["Month"] = jongro["tm"]
jongro["Days"] = jongro["tm"]
jongro["hours"] = jongro["tm"]
days = [int(str(mmdd)[6:8]) for mmdd in jongro["tm"]]
month = [int(str(mmdd)[4:6]) for mmdd in jongro["tm"]]
year = [int(str(mmdd)[0:4]) for mmdd in jongro["tm"]]
hours = [int(str(mmdd)[8:10]) for mmdd in jongro["tm"]]

jongro["Days"] = days
jongro["Year"] = year
jongro["Month"] = month
jongro["hours"] = hours
jongro.head()

,tm,serial,flag,pm10,co2,vocs,noise,temp,humi,pm25,Unnamed: 10,Year,Month,Days,hours
0,201804010000,V10O1611251,1,54,-9999,-9999,41,14.7,55,27.0,NaN,2018,4,1,0
1,201804010001,V10O1611251,1,42,-9999,-9999,47,14.7,55,21.0,NaN,2018,4,1,0
2,201804010002,V10O1611251,1,59,-9999,-9999,41,14.6,56,30.0,NaN,2018,4,1,0
3,201804010003,V10O1611251,1,41,-9999,-9999,44,14.6,56,20.0,NaN,2018,4,1,0
4,201804010004,V10O1611251,1,54,-9999,-9999,43,14.6,56,27.0,NaN,2018,4,1,0


측정소 칼럼을 법정동 이름으로 변경
-------------------------------------------------------------------------------
>  데이터의 통합을 위하여 serial에 기록된 측정소 칼럼을 법정동이름으로 바꾸어 줍니다.

In [7]:
jongro_idx = ['가회동','종로 1,2,3,4가동','교남동','사직동','사직동','혜화동','혜화동','종로 1,2,3,4가동','청운효자동','종로 1,2,3,4가동','종로 1,2,3,4가동','종로 1,2,3,4가동','종로 1,2,3,4가동','종로 1,2,3,4가동','종로 1,2,3,4가동','종로 1,2,3,4가동','종로 1,2,3,4가동','숭인2동','부암동','이화동','천연동','종로 1,2,3,4가동','가회동','종로 1,2,3,4가동','가회동','종로5,6가동','창신3동','창신1동','창신1동','명동','평창동']
jongro_station = ['V10O1611722','V10O1611684','V10O1610252','V10O1611172','V01o1610468','V10O1611645','V10O1611623','V10O1611750','V10O1611255','V10O1610546','V10O1610540','V10O1610542','V10O1610543','V10O1610544','V10O1610545','V10O1610567','V10O1612113','V10O1612106','V10O1611170','V10O1611658','V10O1611289','V10O1611173','V10O1611698','V10O1611145','V10O1611634','V10O1611639','V10O1611251','V10O1611151','V10O1611220','V10O1611887','V10O1611258']

jongro_station_idx = dict(zip(jongro_station, jongro_idx))

for key in jongro_station_idx.keys():
    jongro['serial'] = np.where(jongro['serial'] == key, jongro_station_idx[key], jongro['serial'])

jongro.head()


,tm,serial,flag,pm10,co2,vocs,noise,temp,humi,pm25,Unnamed: 10,Year,Month,Days,hours
0,201804010000,창신3동,1,54,-9999,-9999,41,14.7,55,27.0,NaN,2018,4,1,0
1,201804010001,창신3동,1,42,-9999,-9999,47,14.7,55,21.0,NaN,2018,4,1,0
2,201804010002,창신3동,1,59,-9999,-9999,41,14.6,56,30.0,NaN,2018,4,1,0
3,201804010003,창신3동,1,41,-9999,-9999,44,14.6,56,20.0,NaN,2018,4,1,0
4,201804010004,창신3동,1,54,-9999,-9999,43,14.6,56,27.0,NaN,2018,4,1,0


In [8]:
jongro_copy = jongro

In [9]:
jongro_copy = jongro[['Year','Month','Days','hours','serial','pm10','pm25']]

In [10]:
jongro_copy.head()

,Year,Month,Days,hours,serial,pm10,pm25
0,2018,4,1,0,창신3동,54,27.0
1,2018,4,1,0,창신3동,42,21.0
2,2018,4,1,0,창신3동,59,30.0
3,2018,4,1,0,창신3동,41,20.0
4,2018,4,1,0,창신3동,54,27.0


결측치와 이상치 제거
-------------------------------------------------------------------------------
>  결측치와 이상치를 확인하고 제거해줍니다.

In [11]:
#결측치 확인하기
jongro_copy.isnull().sum()

Year           0
Month          0
Days           0
hours          0
serial         0
pm10           0
pm25      571890
dtype: int64

In [12]:
#이상치 확인하기
jongro_copy[jongro_copy['pm25']< 0].sum()


Year                                               47993682
Month                                                203430
Days                                                 331130
hours                                                242849
serial    창신3동창신3동창신3동창신3동창신3동창신3동창신3동창신3동창신3동창신3동창신3동창신...
pm10                                              -23758218
pm25                                           -2.37582e+07
dtype: object

In [13]:
# 이상치 결측치로 처리하기

jongro_copy.loc[jongro_copy['pm25'] < 0, 'pm25'] = None

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [14]:
jongro_copy.isnull().sum()

Year           0
Month          0
Days           0
hours          0
serial         0
pm10           0
pm25      595672
dtype: int64

In [15]:
jongro_copy.loc[jongro_copy['pm10'] < 0, 'pm10'] = np.nan

In [16]:
jongro_copy.isnull().sum()

Year           0
Month          0
Days           0
hours          0
serial         0
pm10      595672
pm25      595672
dtype: int64

In [17]:
jongro_copy2 = jongro_copy.copy()

In [18]:
jongro_copy2 = jongro_copy2.dropna()

In [19]:
jongro_copy2.isnull().sum()

Year      0
Month     0
Days      0
hours     0
serial    0
pm10      0
pm25      0
dtype: int64

In [20]:
jongro_copy3 = jongro_copy2.groupby(['serial','Year', 'Month','Days','hours'])['pm25','pm10'].mean().astype(int)

In [21]:
jongro_copy3

pm25  pm10
serial Year Month Days hours            
가회동    2018 4     1    0        28    54
                       1        27    56
                       2        27    58
                       3        29    58
                       4        28    54
                       5        25    53
                       6        26    55
                       7        33    64
                       8        31    59
                       9        27    59
                       10       26    64
                       11       26    63
                       12       25    58
                       13       22    51
                       14       26    58
                       15       27    61
                       16       36    70
                       17       56    98
                       18       48    88
                       19       32    67
                       20       39    84
                       21       36    87
                       22       36    89
                       23       36    82
                  2    0        29    65
                       1        26    63
                       2        27    69
                       3        23    61
                       4        25    65
                       5        32    80
...                            ...   ...
혜화동    2019 3     30   18       18    43
                       19       21    39
                       20       32    48
                       21       34    48
                       22       37    52
                       23       33    44
                  31   0        34    44
                       1        33    41
                       2        33    39
                       3        30    40
                       4        32    42
                       5        34    42
                       6        36    44
                       7        43    53
                       8        52    66
                       9        42    61
                       10       29    52
                       11       31    60
                       12       33    63
                       13       29    54
                       14       31    61
                       15       30    61
                       16       35    66
                       17       29    57
                       18       15    37
                       19       15    32
                       20       17    35
                       21       16    36
                       22       16    36
                       23       18    31

[128528 rows x 2 columns]

In [22]:
jongro_copy3.to_csv('pre_jongro.csv')

In [23]:
pre_jongro = pd.read_csv('pre_jongro.csv')
pre_jongro

,serial,Year,Month,Days,hours,pm25,pm10
0,가회동,2018,4,1,0,28,54
1,가회동,2018,4,1,1,27,56
2,가회동,2018,4,1,2,27,58
3,가회동,2018,4,1,3,29,58
4,가회동,2018,4,1,4,28,54
5,가회동,2018,4,1,5,25,53
6,가회동,2018,4,1,6,26,55
7,가회동,2018,4,1,7,33,64
8,가회동,2018,4,1,8,31,59
9,가회동,2018,4,1,9,27,59


법정동 칼럼을 지역코드로 변경
-------------------------------------------------------------------------------
>  데이터의 통합을 위하여 serial에 기록된 법정동 칼럼을 지역코드로 바꾸어 줍니다.

In [24]:
jongro_code = {11110515:'청운효자동', 11110530: '사직동', 11110540: '삼청동', 11110550: '부암동',11110560: '평창동', 11110570: "무악동", 11110580: '교남동', 11110600: '가회동', 11110615: '종로 1,2,3,4가동', 11110630: '종로5,6가동',11110640: "이화동", 11110650: '혜화동', 11110670: '창신1동', 11110680: '창신2동', 11110690: '창신3동', 11110700: '숭인1동', 11110710: '숭인2동'}

for key in jongro_code.keys():
    pre_jongro['serial'] = np.where(pre_jongro['serial'] == jongro_code[key], key, pre_jongro['serial'])


In [25]:
pre_jongro

,serial,Year,Month,Days,hours,pm25,pm10
0,11110600,2018,4,1,0,28,54
1,11110600,2018,4,1,1,27,56
2,11110600,2018,4,1,2,27,58
3,11110600,2018,4,1,3,29,58
4,11110600,2018,4,1,4,28,54
5,11110600,2018,4,1,5,25,53
6,11110600,2018,4,1,6,26,55
7,11110600,2018,4,1,7,33,64
8,11110600,2018,4,1,8,31,59
9,11110600,2018,4,1,9,27,59


In [26]:
#다른 데이터에서 일치하지 않는 행정동 행 제거
pre_jongro = pre_jongro[pre_jongro.serial != '명동']
pre_jongro = pre_jongro[pre_jongro.serial != '천연동']

In [27]:
pre_jongro.to_csv('pre_jongro.csv', index = False)
pre_jongro = pd.read_csv('pre_jongro.csv')
pre_jongro

,serial,Year,Month,Days,hours,pm25,pm10
0,11110600,2018,4,1,0,28,54
1,11110600,2018,4,1,1,27,56
2,11110600,2018,4,1,2,27,58
3,11110600,2018,4,1,3,29,58
4,11110600,2018,4,1,4,28,54
5,11110600,2018,4,1,5,25,53
6,11110600,2018,4,1,6,26,55
7,11110600,2018,4,1,7,33,64
8,11110600,2018,4,1,8,31,59
9,11110600,2018,4,1,9,27,59


In [34]:
pre_jongro['Datetime'] = pre_jongro['Year'].astype('str') + pre_jongro['Month'].astype('str') + pre_jongro['Days'].astype('str')
f1 = lambda x: '%s/%02d/%02d'%(x[1], x[2], x[3])
pre_jongro['Datetime'] = pre_jongro['Datetime'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

In [35]:
pre_jongro.to_csv('pre_jongro.csv', index = False)
pre_jongro = pd.read_csv('pre_jongro.csv')
pre_jongro

,serial,Year,Month,Days,hours,pm25,pm10,Datetime
0,11110600,2018,4,1,0,28,54,2018-04-01
1,11110600,2018,4,1,1,27,56,2018-04-01
2,11110600,2018,4,1,2,27,58,2018-04-01
3,11110600,2018,4,1,3,29,58,2018-04-01
4,11110600,2018,4,1,4,28,54,2018-04-01
5,11110600,2018,4,1,5,25,53,2018-04-01
6,11110600,2018,4,1,6,26,55,2018-04-01
7,11110600,2018,4,1,7,33,64,2018-04-01
8,11110600,2018,4,1,8,31,59,2018-04-01
9,11110600,2018,4,1,9,27,59,2018-04-01
